<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/covid19_tweet/GRU_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0번

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import re

warnings.filterwarnings('ignore')

os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('validation.csv')

rm: cannot remove '*.*': No such file or directory
  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 34.4MB/s]
  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 89.7MB/s]


In [2]:
train_df.head()

,ID,Tweet,Labels
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10
1,2,BanMediaHouse whose is responsible for spreadi...,6
2,3,Im waiting for someone to say to me that all t...,3 4
3,4,He is a liar. Proven day night. Time again. Li...,6
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8


In [3]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [4]:
train_df['label']  = train_df['Labels'].apply(lambda x :  labelsplit(x))
train_df.head(2)

,ID,Tweet,Labels,label
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10,"[0, 10]"
1,2,BanMediaHouse whose is responsible for spreadi...,6,[6]


In [5]:
onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in train_df.label[i] : # j = value   
        onehot[i,j]  = 1

onehot = pd.DataFrame(onehot, columns=['0','1','2','3','4','5','6','7','8','9','10'])
onehot[:3]

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_df = pd.concat(
    [train_df, onehot], axis=1
).drop(['ID','Labels', 'label'],axis=1)

train_df.head(2)

,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [8]:
train_df = train_df[['Tweet','0']]
train_df.head()

,Tweet,0
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0
2,Im waiting for someone to say to me that all t...,0.0
3,He is a liar. Proven day night. Time again. Li...,0.0
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0


In [18]:
!pip install sentencepiece
import sentencepiece as spm

     |████████████████████████████████| 1.2MB 3.9MB/s 


In [20]:
train_df.head()

,Tweet,0
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0
2,Im waiting for someone to say to me that all t...,0.0
3,He is a liar. Proven day night. Time again. Li...,0.0
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0


In [21]:
full_text = np.concatenate(
    [train_df['Tweet'].values, test_df['Tweet'].values]
)

with open('./all_text.txt' ,'w', encoding='utf-8') as f:
    for line in full_text :
        if len(str(line).strip()) == 0 : continue
        try :
            f.write(line + '\n')
        except :
            continue


In [22]:
!ls -alh

total 1.5M
drwxr-xr-x 1 root root 4.0K May 30 03:39 .
drwxr-xr-x 1 root root 4.0K May 30 03:19 ..
-rw-r--r-- 1 root root 710K May 30 03:39 all_text.txt
drwxr-xr-x 4 root root 4.0K May  6 13:43 .config
drwxr-xr-x 1 root root 4.0K May  6 13:44 sample_data
-rw-r--r-- 1 root root 519K May 30 03:23 training.csv
-rw-r--r-- 1 root root 255K May 30 03:23 validation.csv


In [23]:
!cat all_text.txt | head -n10

NO JOKE I WILL HOP ON A PLANE RN! (Well after COVID19 lol)
BanMediaHouse whose is responsible for spreading Fake and communal Stories in this pandemic Corona situation.
Im waiting for someone to say to me that all this corona thing is just an April fools joke
He is a liar. Proven day night. Time again. Lies when the truth will do. COVID19
NEW: U.S. CoronaVirus death toll reaches 4,000 after nearly 900 new deaths were reported today (BNO News) Covid19 CoronaVirusOutbreak
Coronavirus impact Govt extends I-T deadlines related to Sections 80C, 80D
42,000 people might have died in China from Covid19 but China was underreporting according to sources.
Dear Chinese! Kindly cook your bat thoroughly next time. Regards, COVID19 CoronaVirusUpdates CoronaVirus
This is how the govt of kenya is checking the temperatures of COVID19 and saying it is still low. Mtashangaa sana.
My mental health hasn't suffered at all under the coronavirus quarantine! Ha-ha, April Fools.


In [25]:
spm.SentencePieceTrainer.train('--input=all_text.txt --model_prefix=m --vocab_size=11412')

In [26]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [39]:
s = ['It will be good', 'I am a good teacher', 'the good']
for i in s :
    if len(i.split()) < 3 :
        print(i ,':', '\t','\t',sp.encode_as_ids(i) )
        continue
    print(i ,':', '\t',sp.encode_as_ids(i) )


It will be good : 	 [200, 65, 27, 148]
I am a good teacher : 	 [11, 217, 8, 148, 2322]
the good : 	 	 [4, 148]


In [41]:
train_df['bow'] = train_df['Tweet'].apply(lambda x : sp.encode_as_ids(str(x)))
test_df['bow'] = test_df['Tweet'].apply(lambda x : sp.encode_as_ids(str(x)))

In [42]:
train_df.head()

,Tweet,0,bow
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,"[1506, 1526, 11, 1663, 8289, 1225, 80, 6986, 6..."
1,BanMediaHouse whose is responsible for spreadi...,0.0,"[2948, 5938, 8937, 102, 543, 14, 1781, 500, 20..."
2,Im waiting for someone to say to me that all t...,0.0,"[72, 152, 9, 20, 215, 5, 94, 5, 36, 25, 42, 21..."
3,He is a liar. Proven day night. Time again. Li...,0.0,"[395, 14, 8, 2188, 3, 2837, 3480, 96, 531, 3, ..."
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0,"[1190, 38, 340, 3, 141, 3, 280, 187, 442, 772,..."


In [43]:
train_text = train_df['bow']
test_text = test_df['bow']

In [54]:
print(max([len(x) for x in train_text]))
print(min([len(x) for x in train_text]))

44
3


In [55]:
print(max([len(x) for x in test_text]))
print(min([len(x) for x in test_text]))

68
4


In [63]:
train_bow_text = tf.keras.preprocessing.sequence.pad_sequences(train_text, maxlen=70)

In [64]:
train_text.values[:2]

array([list([1506, 1526, 11, 1663, 8289, 1225, 80, 6986, 6759, 45, 174, 8926, 181, 19, 17, 384, 149]),
       list([2948, 5938, 8937, 102, 543, 14, 1781, 500, 20, 495, 3132, 15, 2607, 155, 11036, 18, 21, 158, 23, 544, 3])],
      dtype=object)

In [65]:
train_bow_text.shape

(5000, 70)

In [58]:
train_df['0']

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
4995    0.0
4996    0.0
4997    0.0
4998    0.0
4999    0.0
Name: 0, Length: 5000, dtype: float64

In [71]:
y = keras.utils.to_categorical(train_df['0'])
y[:5]

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [127]:
pd.DataFrame(y)[0].value_counts()

1.0    3820
0.0    1180
Name: 0, dtype: int64

In [72]:
x_train, x_val, y_train, y_val = train_test_split(train_bow_text, y, test_size = 0.3, random_state = 42)

In [75]:
x_train.shape, x_val.shape

((3500, 70), (1500, 70))

In [76]:
y_train.shape, y_val.shape

((3500, 2), (1500, 2))

In [148]:
from keras.layers import Input, Embedding, GRU, Dense, Dropout
from keras.models import Model

def seq2seq() :
    input_x_bow = Input(shape=(70,))
    embedding = Embedding(11412,200)
    x = embedding(input_x_bow)
    #x = Dropout(0.5)(x)
    z = GRU(32)(x)

    y = Dense(2, activation = 'softmax')(z)

    model = Model(input_x_bow, y)
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = 'adam',
        metrics= ['accuracy']
    )
    return model

In [149]:

model = seq2seq()
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 70)]              0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 70, 200)           2282400   
_________________________________________________________________
gru_12 (GRU)                 (None, 32)                22464     
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 66        
Total params: 2,304,930
Trainable params: 2,304,930
Non-trainable params: 0
_________________________________________________________________


In [150]:
hist  = model.fit(x_train, y_train,
                  validation_data = (x_val, y_val),
                  epochs = 3,
                  shuffle = True,
                  class_weight =  {0 : 0.75,
                1: 0.25},
                  
                  verbose = 1)

Epoch 1/3
110/110 [==============================] - 10s 74ms/step - loss: 0.3115 - accuracy: 0.7400 - val_loss: 0.6042 - val_accuracy: 0.7660
Epoch 2/3
110/110 [==============================] - 8s 68ms/step - loss: 0.1791 - accuracy: 0.7442 - val_loss: 0.6643 - val_accuracy: 0.7680
Epoch 3/3
110/110 [==============================] - 8s 69ms/step - loss: 0.0968 - accuracy: 0.8734 - val_loss: 0.7468 - val_accuracy: 0.7520


In [151]:
answer = model.predict(tf.keras.preprocessing.sequence.pad_sequences(test_text, maxlen=70))

In [152]:
Answer = pd.DataFrame(answer, columns =['0','1'])
Answer

,0,1
0,0.997713,0.002287
1,0.999709,0.000291
2,0.409289,0.590711
3,0.999225,0.000774
4,0.998654,0.001346
...,...,...
2495,0.232255,0.767745
2496,0.999834,0.000166
2497,0.694222,0.305778
2498,0.995971,0.004029


In [154]:
Answer['Answer'] = Answer['0'].apply(lambda x : 0 if x >=0.5  else 1) ## else : 1 -> n

In [155]:
Answer.Answer.value_counts()

0    2334
1     166
Name: Answer, dtype: int64

In [156]:
Answer.head()

,0,1,Answer
0,0.997713,0.002287,0
1,0.999709,0.000291,0
2,0.409289,0.590711,1
3,0.999225,0.000774,0
4,0.998654,0.001346,0


In [163]:
ans_0 = pd.DataFrame(Answer['Answer'])

In [166]:
ans_0.columns = ['0']
ans_0.to_csv('')

,0
0,0
1,0
2,1
3,0
4,0
...,...
2495,1
2496,0
2497,0
2498,0


In [ ]:
def auto() :
    